## Module d'estimation de biens immobilier
## Estimation des appartements et des maisons

## 1. Libraries import

In [7]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
import pickle
import requests


## 2. Loading Dataset file

In [27]:
# read file 
df_val_fonc = pd.read_csv('../data/IN/valeursfoncieres-2019.txt', sep = '|',  decimal = ',' , encoding='UTF-8')



## 3. Cleaning and Filtring Dataset

In [28]:
#print(df_val_fonc.head())
#drop empty columns 
df_val_fonc = df_val_fonc.drop(['Code service CH', 'Reference document', '1 Articles CGI', '2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI', 'No disposition','Identifiant local'], axis = 1)
#print(df_val_fonc.head())
#faire un groupby
df_val_fonc = df_val_fonc.groupby(['Date mutation', 'Nature mutation','Valeur fonciere','No voie','Voie','Code postal','Commune','Code departement','No plan','Type local','Code type local', 'Surface terrain'])['Surface reelle bati','Nombre pieces principales'].sum().reset_index()
#
df_val_fonc = df_val_fonc.drop(['Date mutation', 'No voie','No plan','Code type local'], axis = 1)
# # lines  filter
df_val_fonc = df_val_fonc.dropna()
df_val_fonc.drop_duplicates(inplace = True)


KeyError: "['Code service CH' 'Reference document' '1 Articles CGI' '2 Articles CGI'\n '3 Articles CGI' '4 Articles CGI' '5 Articles CGI' 'No disposition'\n 'Identifiant local'] not found in axis"

## 4. Filtering on "Type local" = Apartments

In [ ]:
# récupérere des appartements
df_val_fonc_App = df_val_fonc.loc[(df_val_fonc['Nature mutation']=="Vente") & (df_val_fonc['Type local']=="Appartement"),:]
df_val_fonc_App = df_val_fonc_App.drop(['Nature mutation'], axis = 1)

# filtrer par rapport à la 'Valeur fonciere' et la 'Surface reelle bati'
df_val_fonc_App.drop(df_val_fonc_App[(df_val_fonc_App['Valeur fonciere']< 7000.00)|(df_val_fonc['Valeur fonciere'].isna())].index, inplace = True)
df_val_fonc_App.drop(df_val_fonc_App[df_val_fonc_App['Valeur fonciere']> 20000000.00].index, inplace = True)
#
df_val_fonc_App.drop(df_val_fonc_App[df_val_fonc_App['Surface reelle bati']< 9.00].index, inplace = True)


## 4. Filtering on "Type local" = Houses

In [30]:
# récupérer des maisons
df_val_fonc_maison = df_val_fonc.loc[(df_val_fonc['Nature mutation']=="Vente") & (df_val_fonc['Type local']=="Maison"),:]
df_val_fonc_maison = df_val_fonc_maison.drop(['Nature mutation'], axis = 1)

# filtrer par rapport à la 'Valeur fonciere' et la 'Surface reelle bati'
df_val_fonc_maison.drop(df_val_fonc_maison[(df_val_fonc_maison['Valeur fonciere']< 90000.00)|(df_val_fonc['Valeur fonciere'].isna())].index, inplace = True)
df_val_fonc_maison.drop(df_val_fonc_maison[df_val_fonc_maison['Valeur fonciere']> 15000000.00].index, inplace = True)
#
df_val_fonc_maison.drop(df_val_fonc_maison[df_val_fonc_maison['Surface reelle bati']< 20.00].index, inplace = True)
df_val_fonc_maison.drop(df_val_fonc_maison[df_val_fonc_maison['Nombre pieces principales']> 10].index, inplace = True)

In [ ]:

#
plt.scatter(df_val_fonc_App["Surface reelle bati"], df_val_fonc_App["Valeur fonciere"])
plt.xlabel("Surface batie")
plt.ylabel("Valeur fonciere")
plt.show()

In [ ]:
matrice_corr = df_val_fonc_App.corr().round(3)
sns.heatmap(data = matrice_corr, annot=True)

## Linear regression model

In [ ]:
columns = df_val_fonc_App.columns.tolist()
# Filtrer les colonnes pour supprimer celles que nous ne voulons pas.
columns = [c for c in columns if c not in ["Valeur fonciere","Voie", "Commune","Type local", "Code departement"]]
#stocker la variable à prédire
target = "Valeur fonciere"
# générer l'ensemble de données pour l'apprentissage, définir un état aléatoire pour reproduire les résultats
train = df_val_fonc_App.sample(frac=0.8, random_state=1)

# selectionner un autre ensemble qui n'est pas dans l'apprentissage
test = df_val_fonc_App.loc[~df_val_fonc_App.index.isin(train.index)]
# Initialiser la classe du modèle
lin_model = LinearRegression()
# Ajuster le modèle aux données d'apprentissage (fit the model to the training data)
lin_model.fit(train[columns], train[target])
#lin_model.fit(test[columns], test[target])
# générer les predictions
lin_predictions = lin_model.predict(test[columns]).round(3)
#print("Predictions:", lin_predictions)
print('Values test : ', test[target][:5])
print('Predict Values predict : ', lin_predictions[:5])
# calcul d'erreur entre les prédictions et les valeurs fournies
lin_mse = mean_squared_error(lin_predictions,test[target]).round(3)
print("Computed error:", lin_mse)
#lin_model.score(test[columns],train[target])
lin_model.score(test[columns], test[target])

### Linear Regression model is not adapted to our dataset beacause there is correlation betwween features (Surafce reelle bati & nombre de piece principale).  
###  So we use another model wich regression we shon va essayer l'algorithme random forest car c'est un algorithme qui peut prendre en compte les non linéarités dans les données.

## 5. Selecting Features and spliting the dataset

In [ ]:

# Selecting Features
x = df_val_fonc_App[['Code postal','Surface reelle bati','Nombre pieces principales', 'Surface terrain']]
# Selecting Target 
y = df_val_fonc_App[['Valeur fonciere']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1, shuffle =True)
#
#np.isfinite(X_test).all()


## 6. Fitting random Forest Regressor to the dataset

In [ ]:
model_RF = RandomForestRegressor(n_estimators = 100, min_samples_leaf = 4, random_state = 1)
model_RF.fit(x_train, y_train)
model_RF.fit(x_test, y_test)
#
lin_predictions_FR = model_RF.predict(x_test).round(3)
#
print('Values test : ', y_test[:5])
print('Predict Values predict : ', lin_predictions_FR[:5])
#lin_mse = mean_squared_error(lin_predictions, y)
model_RF.score(x_test, y_test)

## 7. Model Serialization with Pickle


In [ ]:
with open('model_rfg_apartment_pickle','wb') as f:
    pickle.dump(model_RF, f)


##  Random Forest Regressor model application on houses Dataset

In [ ]:
plt.scatter(df_val_fonc_maison["Surface reelle bati"], df_val_fonc_maison["Valeur fonciere"])
plt.xlabel("Surface batie")
plt.ylabel("Valeur fonciere")
plt.show()

In [ ]:
matrice_corr = df_val_fonc_maison.corr().round(3)
sns.heatmap(data = matrice_corr, annot=True)

## .Selecting Features and spliting the dataset

In [31]:
# Selecting Features
X = df_val_fonc_maison[['Code postal','Surface reelle bati','Nombre pieces principales', 'Surface terrain']]
# Selecting Target 
Y = df_val_fonc_maison[['Valeur fonciere']]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1, shuffle =True)
#
#np.isfinite(X_test).all()
Maison_RF = RandomForestRegressor(n_estimators = 100, min_samples_leaf = 4, random_state = 1)
Maison_RF.fit(X_train, Y_train)
Maison_RF.fit(X_test, Y_test)
#
Maison_predictions_RF = Maison_RF.predict(X_test).round(3)
#
print('Values test : ', Y_test[:5])
print('Predict Values predict : ', Maison_predictions_RF[:5])
#lin_mse = mean_squared_error(lin_predictions, y)
Maison_RF.score(X_test, Y_test)

Values test :          Valeur fonciere
59468           94450.0
96303          139500.0
29433          180000.0
482196         390000.0
146440         220000.0
Predict Values predict :  [198549.206 181372.378 186012.306 361010.53  209636.689]


0.5831800044358946

## 7. Model Serialization with Pickle

In [32]:
with open('model_rfg_house_pickle','wb') as f1:
    pickle.dump(Maison_RF, f1)

## Predecting Value With FatsAPI

In [76]:
code_postal = 75010
surafce_reelle = 42
nb_piece = 2
surface_terrain = 100
# maision = 2, appartement =1
type_local = 1

req= {"surafce_reelle":80, "nb_piece": 4, "surface_terrain": 80, "code_postal": 75000, "type_local":1}

response = requests.get("http://localhost:8000/api/estimate/", params=req)
print(response.json())



{'estimation': '1285767.41 €'}
